In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [4]:
!wget  https://www.dropbox.com/s/9kits2euwawcsj0/iris.data.txt

--2020-01-20 13:06:54--  https://www.dropbox.com/s/9kits2euwawcsj0/iris.data.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.1, 2620:100:6016:1::a27d:101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/9kits2euwawcsj0/iris.data.txt [following]
--2020-01-20 13:06:54--  https://www.dropbox.com/s/raw/9kits2euwawcsj0/iris.data.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc6dcd282ae658395859abe42726.dl.dropboxusercontent.com/cd/0/inline/AwjxnmnbgLcjSAnqMs7t3at0Lr6Z2tF5ExJSXnqFhUEtBvpUXXZ57YGBIbcMubN-3Ni2j9d045bfX3_VYadRaq3qb4eU40yl1t9EsZpqRJRk_Q/file# [following]
--2020-01-20 13:06:54--  https://uc6dcd282ae658395859abe42726.dl.dropboxusercontent.com/cd/0/inline/AwjxnmnbgLcjSAnqMs7t3at0Lr6Z2tF5ExJSXnqFhUEtBvpUXXZ57YGBIbcMubN-3Ni2j9d045bfX3_VYadRaq3qb4eU40yl1t9EsZpqRJRk_Q/file
Resolving uc6dcd

In [0]:
from pyspark import SparkContext, SparkConf
from math import sqrt
sc=SparkContext.getOrCreate()

In [0]:

def computeDistance(x,y):
    return sqrt(sum([(a - b)**2 for a,b in zip(x,y)]))
 
 
def closestCluster(dist_list):
    cluster = dist_list[0][0]
    min_dist = dist_list[0][1]
    for elem in dist_list:
        if elem[1] < min_dist:
            cluster = elem[0]
            min_dist = elem[1]
    return (cluster,min_dist)
 
def sumList(x,y):
    return [x[i]+y[i] for i in range(len(x))]
 
def moyenneList(x,n):
    return [x[i]/n for i in range(len(x))]
 
def simpleKmeans(data, nb_clusters):
    clusteringDone = False
    number_of_steps = 0
    current_error = float("inf")
    
    nb_elem = sc.broadcast(data.count())
 

 
    centroides = sc.parallelize(data.takeSample('withoutReplacment',nb_clusters))\
              .zipWithIndex()\
              .map(lambda x: (x[1],x[0][1][:-1]))

#---------------------------------------------------------------OPTIMISATION : BROADCAST-------------------------------------------------------------------------------------------
    sc.broadcast(centroides.collect()) 
  
     
    while not clusteringDone:
 
   
        
        joined = data.cartesian(centroides)
 
        # We compute the distance between the points and each cluster
        dist = joined.map(lambda x: (x[0][0],(x[1][0], computeDistance(x[0][1][:-1], x[1][1]))))
        # (0, (0, 0.866025403784438))
 
        dist_list = dist.groupByKey().mapValues(list)
        # (0, [(0, 0.866025403784438), (1, 3.7), (2, 0.5385164807134504)])
 
        # We keep only the closest cluster to each point.
        min_dist = dist_list.mapValues(closestCluster)
        # (0, (2, 0.5385164807134504))
 
        # assignment will be our return value : It contains the datapoint,
        # the id of the closest cluster and the distance of the point to the centroid
        assignment = min_dist.join(data)
 
        # (0, ((2, 0.5385164807134504), [5.1, 3.5, 1.4, 0.2, 'Iris-setosa']))
 
         
        clusters = assignment.map(lambda x: (x[1][0][0], x[1][1][:-1]))
        clusters.getNumPartitions()
 #----------------------------------------------------------------OPTIMISATION: coalesce-------------------------------------------------------------------------------------------       
        clusters.coalesce(3)

        
 #----------------------------------------------------------------OPTIMISATION: Combine By Key---------------------------------------------------------------------------------------------------------
       
        centroidesCluster=clusters.combineByKey((lambda x:(x,1)),(lambda x,y:(sumList(x[0],y),(x[1]+1))),(lambda x,y: ( sumList(x[0],y[0]),x[1]+y[1])))\
                                  .map(lambda x:((x[0],moyenneList(x[1][0],x[1][1]))))

 #-----------------------------------------------------------------OPTIMISATION : Cache----------------------------------------------------------------------------------------------       
        centroidesCluster.cache()
 #-----------------------------------------------------------------OPTIMISATION: broadcast------------------------------------------------------------------------------------------       
        sc.broadcast(centroidesCluster.collect())                       
        """ 
        count = clusters.map(lambda x: (x[0],1)).reduceByKey(lambda x,y: x+y)
        somme = clusters.reduceByKey(sumList)
        centroidesCluster = somme.join(count).map(lambda x : (x[0],moyenneList(x[1][0],x[1][1])))
        """
        ############################
        # Is the clustering over ? #
        ############################
 
        # Let's see how many points have switched clusters.
        if number_of_steps > 0:
            switch = prev_assignment.join(min_dist)\
                                    .filter(lambda x: x[1][0][0] != x[1][1][0])\
                                    .count()
        else:
            switch = 150

        print('step',number_of_steps ,'-->','Num points have switched clusters: ',switch)

        if switch == 0 or number_of_steps == 100:
            clusteringDone = True
            error = sqrt(min_dist.map(lambda x: x[1][1]).reduce(lambda x,y: x + y))/nb_elem.value
        else:
            centroides = centroidesCluster
            prev_assignment = min_dist
            number_of_steps += 1
    print('number of steps finale: ',number_of_steps)
 
    return (assignment, error, number_of_steps)

In [0]:
lines=sc.textFile('iris.data.txt')

In [0]:
data = lines.map(lambda x: x.split(','))\
 .map(lambda x: [float(i) for i in x[:4]] + [x[4]])\
 .zipWithIndex()\
 .map(lambda x: (x[1], x[0])) 

In [13]:
%%time
clustering = simpleKmeans(data,3)


step 0 --> Num points have switched clusters:  150
step 1 --> Num points have switched clusters:  29
step 2 --> Num points have switched clusters:  6
step 3 --> Num points have switched clusters:  3
step 4 --> Num points have switched clusters:  1
step 5 --> Num points have switched clusters:  0
number of steps finale:  5
CPU times: user 834 ms, sys: 159 ms, total: 994 ms
Wall time: 1min 51s
